In [1]:
import pandas as pd
from text2vec import SentenceModel
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.models import PointStruct
import tqdm

/Users/tianlimin/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
#os.chdir('/root/ChatBot/QA')
print(os.getcwd())

/Users/tianlimin/vector based model/data_import


In [3]:
# 加载向量化模型
t2v_model = SentenceModel("/Users/tianlimin/LLM/model-parameter/text2vec_cmed")

2024-02-03 22:08:47.634 | DEBUG    | text2vec.sentence_model:__init__:80 - Use device: cpu


In [4]:
def to_embeddings(text):
    sentence_embeddings = t2v_model.encode(text)
    return sentence_embeddings

vec = to_embeddings('肚子痛拉稀两天今天三次怎么回事')
vec.shape

(768,)

In [5]:
#连接向量数据库
with open('key.txt') as f:
    api_key = f.read()
client = qdrant_client = QdrantClient(
    url="https://635ebebf-42a7-4833-8d6c-83d8d7685d5f.us-east4-0.gcp.cloud.qdrant.io", 
    api_key=api_key,
)
collection_name = "questions"
# 创建collection
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

True

In [6]:
data = pd.read_csv('source_data/admin 3.csv',delimiter = ',', encoding='utf8')
data.head()

,id,text,label,Comments
0,1944,"Value statement：As a site visitor, I want to b...",Value Statement Score:\n\nOne role defined (10...,NaN
1,1945,"Value statement: As an event organiser, I want...",Value Statement Score:\n\nOne role defined (10...,NaN
2,1946,"Value statement: As a content editor, I want t...",Value Statement Score:\n\nOne role defined (10...,NaN
3,1947,"Value statement: As a content editor, I want t...",Value Statement Evaluation:\nOne role defined:...,NaN
4,1948,"Value statement: As a content editor, I want t...",Value Statement Evaluation:\nOne role defined:...,NaN


In [7]:
data['label']

0     Value Statement Score:\n\nOne role defined (10...
1     Value Statement Score:\n\nOne role defined (10...
2     Value Statement Score:\n\nOne role defined (10...
3     Value Statement Evaluation:\nOne role defined:...
4     Value Statement Evaluation:\nOne role defined:...
5     Value Statement Evaluation:\nOne role defined:...
6     Value Statement Evaluation:\nOne role defined:...
7     Value Statement Evaluation:\nOne role defined:...
8     Value Statement Evaluation:\nOne role defined:...
9     Value Statement Evaluation:\nOne role defined:...
10    Value Statement Evaluation:\nOne role defined:...
11    Value Statement Evaluation:\nOne role defined:...
12    Value Statement Evaluation:\nOne role defined:...
13    Value Statement Evaluation:\nOne role defined:...
14    Value Statement Evaluation:\nOne role defined:...
15    Value Statement Evaluation:\nOne role defined:...
16    Value Statement Evaluation:\nOne role defined:...
17    Value Statement Evaluation:\nOne role defi

In [40]:
#导入数据
count = 0

for i in tqdm.tqdm(data.index):
    text = '标题：%s||问题：%s||答案：%s' % (data['title'][i],data['ask'][i],data['answer'][i])
    vec = to_embeddings(text)
    client.upsert(
        collection_name=collection_name,
        wait=True,
       points=[
            PointStruct(id=count, vector=vec.tolist(), payload={"department":data['department'][i],"title": data['title'][i], "text": text}),
        ],
    )
    count+=1

  0%|          | 342/115991 [02:55<16:29:46,  1.95it/s]


KeyboardInterrupt: 

In [8]:
count=1

for i in tqdm.tqdm(data.index[:1000]):
    vec = to_embeddings(data['text'][i])
    client.upsert(
        collection_name=collection_name,
        wait=True,
        points=[
            PointStruct(id=count, vector=vec.tolist(), payload={"text":'question：%s || answer：%s' % (data['text'][i],data['label'][i])}),
        ],
    )
    count += 1

100%|███████████████████████████████████████████| 50/50 [00:20<00:00,  2.42it/s]


In [9]:
client.get_collection(collection_name="questions")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=4, indexed_vectors_count=0, points_count=4, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})